In [39]:
from functools import partial
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.datasets import load_sample_image
from matplotlib import pyplot as plt

In [40]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_test = (X_test - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std

X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]

In [41]:
X_train.shape

(55000, 28, 28, 1)

In [42]:
DefaultConv2d = partial(keras.layers.Conv2d, kernel_size=3, activation='relu', padding='SAME')

model = keras.models.Sequential([
  DefaultConv2d(filters=64, kernel_size=7, input_shape=[28,28,1]),
  keras.layers.MaxPooling2d(pool_size=2),
  DefaultConv2d(filters=128),
  DefaultConv2d(filters=128),
  keras.layers.MaxPooling2d(pool_size=2),
  DefaultConv2d(filters=256),
  DefaultConv2d(filters=256),
  keras.layers.MaxPooling2d(pool_size=2),
  keras.layers.Flatten(),
  keras.layers.Dense(units=128, activation='relu'),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(units=64, activation='relu'),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(units=10, activation='softmax')
])

AttributeError: ignored

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer='nadam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
score = model.evaluate(X_test, y_test)
X_new = X_test[:10]
y_pred = model.predict(X_new)

In [ ]:
model2 = keras.applications.resnet50.ResNet50(weights='imagenet')

In [ ]:
def plot_color_image(image):
  plt.imshow(image, interpolation='nearest')
  plt.axis("off")

In [ ]:
china = load_sample_image("china.jpg") / 255
flower = load_sample_image("flower.jpg") / 255
images = np.array([china, flower])

image_resized = tf.image.resize(images, [224,224])
plot_color_image(images_resized[0])
plt.show()

In [ ]:
plot_color_image(china)

In [ ]:
images_resized = tf.image.resize_with_pad(images, 244, 244, antialias=True)
plot_color_image(images_resized[0])
plt.show()

In [ ]:
images_resized = tf.image.resize_with_crop_or_pad(images, 244, 244)
plot_color_image(images_resized[0])
plt.show()

In [ ]:
china_box = [0,0.03,1,0.68]
flower_box = [0.19,0.26,0.86,0.7]
images_resized = tf.image.crop_and_resize(images, [china_box, flower_box], [0,1],[224,224])
plot_color_image(images_resized[0])
plt.show()
plot_color_image(images_resized[1])
plt.show()

In [ ]:
inputs = keras.applications.resnet50.preprocess_input(images_resized * 255)
Y_proba = model2.predict(inputs)

In [ ]:
top_K = keras.applications.resnet50.decode_predictions(Y_proba, top=5)
for image_index in range(len(images)):
  print("Image #{}".format(image_index))
  for class_id, name, Y_proba in top_K[image_index]:
    print("{} - {:12s} {:.2f}%".format(class_id, name, Y_proba * 100))
  print()
                                  

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

In [ ]:
class_names = info.features["label"].class_names
class_names

In [ ]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load("tf_flowers", split=["train[:10%]", "train[10%:25%", "train[25%:]"], as_supervised=True)

In [ ]:
plt.figure(figsize=(12,10))
index = 0
for image, label in train_set_raw.take(9):
  index += 1
  plt.subplot(3,3,index)
  plt.imshow(image)
  plt.title("Class: {}".format(class_names[label]))
  plt.axis("off")

plt.show()

In [ ]:
n_classes = info.features["label"].num_classes
print(n_classes)
dataset_size = info.splits["train"].num_examples
print(dataset_size)

In [ ]:
info

In [ ]:
def central_crop(image):
  shape = tf.shape.(image)
  min_dim = tf.reduce_min([shape[0], shape[1]])
  top_crop = (shape[0] - min_dim)
  bottom_crop = shape[0] - top_crop
  left_crop = (shape[1] - min_dim)
  right_crop = shape[1] - left_crop
  return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
  shape = tf.shape(image)
  min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
  return tf.image.random_crop(image, [min_dim, min_dim, 3])

def preprocess(image, label):
  if randomize:
    cropped_image = random_crop(image)
    cropped_image = tf.image.random_flip_left_right(cropped_image)
  else:
    cropped_image = central_crop(image)

  resized_image = tf.image.resize(image, [299,299])
  final_image = keras.applications.xception.preprocess_input(resized_image)
  return final_image, label

In [ ]:
batch_size = 32
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess).batch(batch_size).prefetch(1))
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [ ]:
plt.figure(figsize=(12,12))
for X_batch, y_batch in train_set.take(1):
  for index in range(9):
    plt.subplot(3,3,index+1)
    plt.imshow(X_batch[index] / 2 + .5)
    plt.title("class: {}".format(class_names[y_batch[index]]))
    plt.axis("off")

plt.show()

In [ ]:
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = keras.layers.GlobalAveragePooling2d()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [ ]:
for index, layer in enumerate(model.layers):
  print(index, layer.name)

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

history = model.fit(train_set, steps_per_epoch=int(dataset_size / batch_size * 0.75), validation_data = valid_set, validation_steps=int(0.15 * dataset_size / batch_size), epochs=5)